In [89]:
%config IPCompleter.greedy=True
from math import isnan, sqrt
import matplotlib.pyplot as plt

import numpy as np
import scipy.optimize as optim

from NuMPI.Tools import Reduction
from inspect import signature

from SurfaceTopography import Topography
from SurfaceTopography import make_sphere
import ContactMechanics as Solid
from scipy.optimize import OptimizeResult
from ContactMechanics import FreeFFTElasticHalfSpace
from ContactMechanics.Systems import NonSmoothContactSystem

In [93]:
nx,ny,Es = 16,16,1.
sx = sy = 6.
R = 3.
substrate = FreeFFTElasticHalfSpace(
    (nx, ny),
    young=Es,
    physical_sizes=(sx, sy))
#communicator=comm,
#fft="mpi")

surface = make_sphere(
    R, (nx, ny), (sx, sy),
    centre=(sx / 2., sy / 2.),
    subdomain_locations=substrate.topography_subdomain_locations,
    nb_subdomain_grid_pts=substrate.topography_nb_subdomain_grid_pts,
    kind="paraboloid" )
np.savetxt('sphere',surface.heights(), fmt='%1.5f')
init_disp = np.zeros(substrate.nb_subdomain_grid_pts)
system = NonSmoothContactSystem(substrate, surface)

penetration = 0.5
lbounds = system._lbounds_from_heights(penetration)
bounded = init_disp < lbounds
init_disp[bounded.filled(False)] = lbounds[bounded.filled(False)]

res = constrained_conjugate_gradients(
    system.objective(penetration, gradient=True),
    # We also test that the logger and the postprocessing involved work properly in parallel
    system.hessian_product,
    init_disp[substrate.subdomain_slices].reshape(-1),
    gtol=1e-13,
    maxiter=1)

(array([134, 135, 136, 137, 138, 165, 166, 167, 168, 169, 170, 171, 196,
       197, 198, 199, 200, 201, 202, 203, 204, 228, 229, 230, 231, 232,
       233, 234, 235, 236, 260, 261, 262, 263, 264, 265, 266, 267, 268,
       292, 293, 294, 295, 296, 297, 298, 299, 300, 324, 325, 326, 327,
       328, 329, 330, 331, 332, 357, 358, 359, 360, 361, 362, 363, 390,
       391, 392, 393, 394]),)


In [91]:
def constrained_conjugate_gradients(fun, hessp, x0, gtol=1e-8,
                                    mean_value=None, residual_plot=True,
                                    maxiter=5000):
    """
    Implementation of constrained conjugate gradient algorithm as described in,
    I.A. Polonsky, L.M. Keer, Wear 231, 206 (1999).

    Parameters
    ----------
    fun : callable.
                The objective function to be minimized.
                            fun(x) -> float(energy),ndarray(gradient)
                where x is the input ndarray.
                Note that energy is never used, you can return a dummy value.

    hessp : callable
            Function to evaluate the hessian product of the objective.
            Hessp should accept either 1 argument (desscent direction) or
            2 arguments (x,descent direction).
                            hessp(des_dir)->ndarray
                                    or
                            hessp(x,des_dir)->ndarray
            where x is the input ndarray and des_dir is the descent direction.

    x0 : ndarray
         Initial guess.
         ValueError is raised if "None" is provided.

    gtol : float, optional
           Default value : 1e-8
           convergence criterion is max(abs) and norm2 of the projected
           gradient < gtol.

    mean_value :  float, optional
               If you want to apply the mean_value constraint then provide an
               float value to the mean_value.

    residual_plot : bool, optional
                    Generates a plot between the residual and iterations.

    maxiter : int, optional
              Default, maxiter=5000
              Maximum number of iterations after which the program will exit.

    Returns
    -------
    OptimizeResult  : scipy.optimize object.
        Attributes:
         success: bool
         x: x,
         jac: residual = gradient(x),
         nit: n_iterations,
         message: 'CONVERGENCE: NORM_OF_GRADIENT_<=_GTOL' or 'NO CONVERGENCE: MAXITERATIONS REACHED'

    References
    ----------
    ..[1] I.A. Polonsky, L.M. Keer, Wear 231, 206 (1999)
    """

    if not isinstance(mean_value, (type(None), int, float)):
        raise ValueError('Inappropriate type: {} for mean_value whereas a '
                         'float \
            or int is expected'.format(type(mean_value)))

    if not isinstance(residual_plot, bool):
        raise ValueError('Inappropriate type: {} for "residual_plot" whereas '
                         'a bool \
                         is expected'.format(type(residual_plot)))

    
    if x0 is not None:
        x = x0.copy()
        x = x.flatten()
        delta = 0
        G_old = 1
    else:
        raise ValueError('Input required for x0/initial value !!')

    np.savetxt('x0', x.reshape(32,32), fmt='%1.4f')
    gaps = np.array([])
    iterations = np.array([])

    des_dir = np.zeros(np.shape(x))

    if residual_plot:
        gaps = np.append(gaps, 0)
        iterations = np.append(iterations, 0)

    n_iterations = 1
    
    while (n_iterations < maxiter + 1):

        '''Mask to truncate the negative values'''
        mask_neg = x <= 0
        x[mask_neg] = 0.0

        '''Initial residual or GAP'''
        residual = fun(x)[1]
        np.savetxt('residual', residual.reshape(32,32))
        mask_c = x > 0
        if mean_value is not None:
            residual = residual - np.mean(residual[mask_c])

        #np.savetxt('residual2', residual.reshape(32,32))
        print(np.where(mask_c!=0))
        G = np.sum(residual[mask_c] ** 2)

        des_dir[mask_c] = -residual[mask_c] + delta * (G / G_old) * des_dir[
            mask_c]
        des_dir[np.logical_not(mask_c)] = 0
        G_old = G

        '''Calculating step-length alpha'''

        sig = signature(hessp)
        if len(sig.parameters) == 2:
            hessp_val = hessp(x, des_dir)
        elif len(sig.parameters) == 1:
            hessp_val = hessp(des_dir)
        else:
            raise ValueError('hessp function has to take max 1 arg (descent '
                             'dir) or 2 args (x, descent direction)')

        '''Here hessp_val is used as r_ij in original algorithm'''
        if mean_value is not None:
            hessp_val = hessp_val - np.mean(hessp_val[mask_c])

        if mask_c.sum() != 0:
            '''alpha is TAU from algorithm'''
            alpha = -np.sum(residual[mask_c] * des_dir[mask_c]) / np.sum(hessp_val[mask_c] * des_dir[mask_c])
        else:
            # TODO: does anything happen when alpha is 0 or is the algorithm just stuck ?
            alpha = 0.0

        if alpha < 0:
            print("it {} : hessian is negative along the descent direction. "
                  "You will probably need linesearch "
                  "or trust region".format(n_iterations))

        x[mask_c] += alpha * des_dir[mask_c]

        '''mask for contact'''
        mask_neg = x <= 0
        '''truncating negative values'''
        x[mask_neg] = 0.0

        mask_g = residual < 0
        mask_overlap = np.logical_and(mask_neg, mask_g)

        if mask_overlap.sum() == 0:
            delta = 1
        else:
            delta = 0
            x[mask_overlap] = x[mask_overlap] - alpha * residual[mask_overlap]

        if mean_value is not None:
            '''Take care of constraint a_x*a_y*sum(p_ij)=P0'''
            P = np.mean(x)
            x *= (mean_value / P)

        if residual_plot:
            iterations = np.append(iterations, n_iterations)
            if mask_c.sum() != 0:
                gaps = np.append(gaps, np.max(abs(residual[mask_c])))
            else:
                gaps = np.append(gaps, np.max(abs(residual)))

        n_iterations += 1
        res_convg = False
        assert np.logical_not(np.isnan(x).any())

        if n_iterations >= 3:
            '''If converged'''
            if mask_c.sum() != 0:
                if np.max(abs(residual[mask_c])) <= gtol:
                    res_convg = True
                else:
                    res_convg = False

            if res_convg:
                result = OptimizeResult(
                    {
                        'success': True,
                        'x': x,
                        'jac': residual,
                        'nit': n_iterations,
                        'message': 'CONVERGENCE: NORM_OF_GRADIENT_<=_GTOL',
                        })
                if residual_plot:
                    import matplotlib.pyplot as plt
                    plt.plot(iterations, np.log10(gaps))
                    plt.xlabel('iterations')
                    plt.ylabel('residuals')
                    plt.show()

                return result

            elif (n_iterations >= maxiter - 1):
                '''If no convergence'''
                result = OptimizeResult({
                    'success': False,
                    'x': x,
                    'jac': residual,
                    'nit': n_iterations,
                    'message': 'NO-CONVERGENCE: MAXITERATIONS REACHED',
                    })

                if residual_plot:
                    import matplotlib.pyplot as plt
                    plt.plot(iterations, np.log10(gaps))
                    plt.xlabel('iterations')
                    plt.ylabel('residuals')
                    plt.show()

                return result

In [92]:
n = 128

obj = MPI_Quadratic((n,n), pnp=Reduction(), )

xstart = np.random.normal(size=obj.nb_subdomain_grid_pts)
#print(obj.hessian_product)

res = constrained_conjugate_gradients(
    obj.f_grad,
    obj.hessian_product,
    x0=xstart,
)
"""assert res.success, res.message
print(res.nit)"""

NameError: name 'MPI_Quadratic' is not defined

In [ ]:
class MPI_Quadratic():
    """
    n should be even

    :param x: 1d array
    :return:
    """
    bounds = (-4, 4)

    def __init__(self, nb_domain_grid_pts, pnp=Reduction(), factors=None,
                 startpoint=None, xmin=None):

        comm = pnp.comm
        nprocs = comm.Get_size()
        rank = comm.Get_rank()

        step = nb_domain_grid_pts // nprocs

        if rank == nprocs - 1:
            self.subdomain_slices = slice(rank * step, None)
            self.subdomain_locations = rank * step
            self.nb_subdomain_grid_pts = nb_domain_grid_pts - rank * step
        else:
            self.subdomain_slices = slice(rank * step, (rank + 1) * step)
            self.subdomain_locations = rank * step
            self.nb_subdomain_grid_pts = step

        # helps to select the data that has odd or even index in the global
        # array
        self.pnp = pnp

        if xmin is None:
            self._xmin = np.zeros(self.nb_subdomain_grid_pts)
        else:
            self._xmin = xmin[self.subdomain_slices]

        if factors is not None:
            self.factors = factors[self.subdomain_slices]
        else:
            self.factors = np.random.random(self.nb_subdomain_grid_pts) + 0.1

        if startpoint is not None:
            self._startpoint = startpoint[self.subdomain_slices]
        else:
            self._startpoint = np.random.normal(
                size=self.nb_subdomain_grid_pts)

        self.nfeval = 0
        self.ngradeval = 0

    def f_grad(self, x):
        self.nfeval += 1
        self.ngradeval += 1
        factdotx = self.factors.reshape(x.shape) \
            * (x - self._xmin.reshape(x.shape))
        return self.pnp.sum(factdotx ** 2, axis=0).item(), 2 * factdotx

    def f(self, x):
        self.nfeval += 1
        factdotx = self.factors.reshape(x.shape) \
            * (x - self._xmin.reshape(x.shape))
        return self.pnp.sum(factdotx ** 2, axis=0).item()

    def grad(self, x):
        self.ngradeval += 1
        return 2 * self.factors.reshape(x.shape) * (x - self._xmin)

    def hessian_product(self, p):
        return 2 * self.factors.reshape(p.shape) * p

    def startpoint(self):
        """
        standard starting point
        :param n:
        :return: array of shape (1,n)
        """
        return self._startpoint

    @staticmethod
    def minVal(*args):
        return 0

    def xmin(self):
        """
        Location of minimum according to
        :param n: number of DOF
        :return: array of size n
        """
        return self._xmin

In [ ]:
import numpy as np

from mpi4py import MPI


def get_dtype_info(dtype):
    if dtype.kind == 'i':
        return np.iinfo(dtype)
    if dtype.kind == 'f':
        return np.finfo(dtype)
    raise ValueError


class Reduction:
    def __init__(self, comm=None):
        if comm is None:
            self.comm = MPI.COMM_SELF
        else:
            self.comm = comm

    def _op(self, npop, npargs, mpiop, *args, **kwargs):
        """
        Generic reduction operation

        Parameters
        ----------
        npop : func
            Numpy reduction function (e.g. np.sum)
        npargs: tuple
            Arguments passed to the reduction function (e.g. array to be
            reduced)
        mpiop : mpi4py.MPI.op
            MPI reduction operation

        Returns
        -------
        result_arr : np.ndarray
            Result of the reduction operation
        """
        local_result = npop(*npargs, *args, **kwargs)
        result = np.zeros_like(local_result)
        mpitype = MPI._typedict[local_result.dtype.char]
        self.comm.Allreduce([local_result, mpitype], [result, mpitype], op=mpiop)
        return result

    def _op1(self, npop, arr, mpiop, *args, **kwargs):
        """
        Generic reduction operation that takes a single (array) argument

        Parameters
        ----------
        npop : func
            Numpy reduction function (e.g. np.sum)
        arr : array_like
            Numpy array containing the data to be reduced
        mpiop : mpi4py.MPI.op
            MPI reduction operation
        initial : arr.dtype
            Value to use if local array is empty

        Returns
        -------
        result_arr : np.ndarray
            Result of the reduction operation
        """
        if 'initial' in kwargs and isinstance(arr, np.ma.MaskedArray):
            # Max/min on masked array do not support `initial`
            arr = arr.filled(kwargs['initial'])
            del kwargs['initial']
        return self._op(npop, (arr,), mpiop, *args, **kwargs)

    def sum(self, arr, *args, **kwargs):
        """
        Summation

        Parameters
        ----------
        arr : array_like
            Numpy array containing the data to be reduced

        Returns
        -------
        result_arr : np.ndarray
            Sum of all elements of the array over all processors
        """
        return self._op1(np.sum, arr, MPI.SUM, *args, **kwargs)

    def max(self, arr, *args, **kwargs):
        """
        Maximum value

        Parameters
        ----------
        arr : array_like
            Numpy array containing the data to be reduced

        Returns
        -------
        result_arr : np.ndarray
            Maximum of all elements of the array over all processors
        """
        kwargs['initial'] = get_dtype_info(arr.dtype).min
        return self._op1(np.max, arr, MPI.MAX, *args, **kwargs)

    def min(self, arr, *args, **kwargs):
        """
        Minimum value

        Parameters
        ----------
        arr : array_like
            Numpy array containing the data to be reduced

        Returns
        -------
        result_arr : np.ndarray
            Minimum of all elements of the array over all processors
        """
        kwargs['initial'] = get_dtype_info(arr.dtype).max
        return self._op1(np.min, arr, MPI.MIN, *args, **kwargs)

    def mean(self, arr, *args, **kwargs):
        """
        Arithmetic mean

        Parameters
        ----------
        arr : array_like
            Numpy array containing the data to be reduced

        Returns
        -------
        result_arr : np.ndarray
            Arithmetic mean of all elements of the array over all processors
        """
        return self.sum(arr, *args, **kwargs) / self.sum(np.ones_like(arr), *args, **kwargs)

    def dot(self, a, b, *args, **kwargs):
        """
        Scalar product a.b

        Parameters
        ----------
        a : array_like
            Numpy array containing the data of the first array
        a : array_like
            Numpy array containing the data of the second array

        Returns
        -------
        result_arr : np.ndarray
            Scalar product between a and b
        """
        return self._op(np.dot, (a, b), MPI.SUM, *args, **kwargs)

    def any(self, arr, *args, **kwargs):
        """
        Returns true of any value is true

        Parameters
        ----------
        arr : array of bools
            Input data

        Returns
        -------
        result_arr : np.ndarray
            True if any value in `arr` is true
        """
        return self._op1(np.any, arr, MPI.LOR, *args, **kwargs)

    def all(self, arr, *args, **kwargs):
        """
        Returns true of all values are true

        Parameters
        ----------
        arr : array of bools
            Input data

        Returns
        -------
        result_arr : np.ndarray
            True if all values in `arr` are true
        """
        return self._op1(np.all, arr, MPI.LAND, *args, **kwargs)